In [1]:
import cantera as ct
import numpy as np
import os
import sys

print('Using the following versions of: ')
print("python",sys.version)
print('numpy',np.__version__)
print('cantera',ct.__version__)


Using the following versions of: 
python 3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
numpy 1.15.4
cantera 2.4.0


##### This code compares net reaction rate computation of TChem++ with cantera

In [2]:
def runTchem():
    os.chdir(working_dirr) 
    os.system('./delete') # delete old files
    # chem files
    chemfile      = dirInputs + 'chem.inp'
    thermfile     = dirInputs + 'therm.dat'
    chemSurfFile  = dirInputs + 'chemSurf.inp'
    thermSurffile = dirInputs + 'thermSurf.dat'
    #operating condition files
    inputfile     = dirInputs + 'inputGas.dat'
    inputfileSurf = dirInputs + 'inputSurfGas.dat'
    #output files
    outputfile    = 'omega.dat'
    outputfileGasSurf = 'omegaGasSurf.dat'
    outputfileSurf    = 'omegaSurf.dat'
    
    comand = direx+"TChem_NetProductionRateSurfacePerMass.x"+\
              " --chemfile="+ chemfile + \
              " --thermfile="+ thermfile +\
              " --chemSurffile=" + chemSurfFile+ \
              " --thermSurffile="+thermSurffile + \
              " --inputfile="+inputfile +\
              " --inputfileSurf=" + inputfileSurf + \
              " --outputfile=" + outputfile +\
              " --outputfileGasSurf=" + outputfileGasSurf +\
              " --outputfileSurf=" + outputfileSurf
#     print(comand)
    os.system(comand)
    
#     os.chdir('cantera')
    return

def printRoP():
    dirOputs = working_dirr +'/'
    species_names =  gas.species_names + surf.species_names
    ## gas 
    omega = np.loadtxt(dirOputs+'omega.dat')
    #surf 
    omegaGasSurf = np.loadtxt(dirOputs+'omegaGasSurf.dat')
    omegaSurf    = np.loadtxt(dirOputs+'omegaSurf.dat')
    omegaS = np.vstack((omegaGasSurf,omegaSurf))
    print('--Surface phase --')
    for i,species in enumerate(species_names):
        print(species,surf.net_production_rates[i],omegaS[i,1])
        
    print('--Surface rate of progress--')
    for i,reac in enumerate(surf.reaction_equations()):
        print(reac,surf.forward_rates_of_progress[i],surf.reverse_rates_of_progress[i])
        
    print('--- reaction constatns ---')
    for i,reac in enumerate(surf.reaction_equations()):
        print(reac,surf.reverse_rate_constants[i],surf.forward_rate_constants[i])
    print('--Gas phase --') 
    for i,species in enumerate(gas.species_names):
        print(species,gas.net_production_rates[i],omega[i,1])
   
    return 


def compareTchemAndCantera():
    dirOputs = working_dirr +'/'
    net_prodGasPhase  = np.loadtxt(dirOputs+'omega.dat') [:,1] 
    net_prodGasSurf   = np.loadtxt(dirOputs+'omegaGasSurf.dat')[:,1] 
    net_prodSurfSurf  = np.loadtxt(dirOputs+'omegaSurf.dat')[:,1] 
    net_prodSurf = np.hstack((net_prodGasSurf,net_prodSurfSurf))
    
    print('-----------------------------')
    print('Gas phase')
    print('species name cantera  Tchem diff')
    print('-----------------------------')
    for i,species in enumerate(gas.species_names):
        print(species,gas.net_production_rates[i],net_prodGasPhase[i],
              gas.net_production_rates[i]-net_prodGasPhase[i],
             (gas.net_production_rates[i]-net_prodGasPhase[i])/(gas.net_production_rates[i]+1e-23))
    
        if abs((gas.net_production_rates[i]-net_prodGasPhase[i])/gas.net_production_rates[i]) > allow_error:
            print('CHECK this rate')
        print('    ')
        
    print('-----------------------------')
    print('Surface interface') 
    print('species, name, cantera,  Tchem,  diff, % diff')
    print('-----------------------------')
    
    species_names =  gas.species_names + surf.species_names
    for i,species in enumerate(species_names):
        print(species,surf.net_production_rates[i],
              net_prodSurf[i],surf.net_production_rates[i]-net_prodSurf[i],
             (surf.net_production_rates[i]-net_prodSurf[i])/(surf.net_production_rates[i]+1e-23))
    
        if abs((surf.net_production_rates[i]-net_prodSurf[i])/surf.net_production_rates[i]) > allow_error:
            print('CHECK this rate')
    
        print('    ')
    return 

 
def getOriginalInput():
    #Reading mass-fraction:
    #======================
    state = np.loadtxt(dirInputs+'inputGasOriginal.dat') # ρ,P,temp, mass fraction
                                  
    Ykk = np.hstack((np.atleast_1d(state[2]),np.atleast_1d(state[1]),state[3:]))
    #Reading site-fraction:
    #======================
    Zkk = np.loadtxt(dirInputs+'inputSurfGasOriginal.dat')
    return Ykk , Zkk

def saveInputsTChem():
    state =  np.hstack((np.atleast_1d(gas.density),np.atleast_1d(Ykk[1]),np.atleast_1d(Ykk[0]),Ykk[2:]))
    # T, P
    np.savetxt(dirInputs + 'inputGas.dat',state)
    np.savetxt(dirInputs + 'inputSurfGas.dat',Zkk)
    return 

def getRandomSample(Nvars,μ=0,σ=1,sum1=True):
    sample = np.random.rand(Nvars)*σ+μ
    if sum1:
        sample /= np.sum(sample)
    
    return sample

def makeTChemRandomSample():
    massFraction = getRandomSample(gas.n_species)
    T            = getRandomSample(1,μ=1200,σ=500,sum1=False)
    P            = 1013250.0 # Pascal
    Ykk = np.hstack((T,P,massFraction))
    Zkk = getRandomSample(surf.n_species)  
    return Ykk,Zkk

### User inputs
* dirInputs : input and/or ouput 
* direx: Where TChem executable (tchem-example-reactionrates-surface.x) is located. source code is at <Tchemrepo>/src/example/TChem_ReactionRatesSurface.cpp
* dirr : run directory     
* allow_error: allow error between cantera and Tchem
* make_random_sample: true or false, if true a random state vector will be used. else a vector will be loaded inputGasOriginal.dat and inputSurfGasOriginal.dat.  

In [3]:
TchemRepo = '/Users/odiazib/csp_clang_bld_develop/TChem++'
working_dirr      = os.getcwd()#TchemRepo + '/master/src/example/SurfVerification/'

direx     = TchemRepo + '/install/example/'
allow_error        = 1e-3
make_random_sample = True
print('Working directory : ',working_dirr)

Working directory :  /Users/odiazib/csp_clang_bld_develop/TChem++/master/src/example/SurfVerification


### With an initial guess

In [4]:
# dirInputs = TchemRepo + '/master/src/example/SurfVerification/data/reaction-rates-surfaces/PT/'
dirInputs = working_dirr + '/data/reaction-rates-surfaces/PT/'
gas  = ct.Solution(dirInputs+'chem.cti', 'gas')
surf = ct.Interface(dirInputs+'chem.cti', 'surface1', [gas])

print('Input directory : ', dirInputs)


# dirInputs = TchemRepo + 'build/example/SurfVerification/data/reaction-rates-surfaces/X/'

# dirInputs = TchemRepo + '/master/src/example/SurfVerification/data/reaction-rates-surfaces/X/'
# gas  = ct.Solution(dirInputs+'chem_annotated-gas_surface.cti', 'gas')
# surf = ct.Interface(dirInputs+'chem_annotated-gas_surface.cti', 'surface1', [gas])

if make_random_sample:
    Ykk , Zkk = makeTChemRandomSample()
else:    
    Ykk , Zkk = getOriginalInput()
#     Ykk , Zkk = getOriginalInput()

gas.Y  = Ykk[2:]
gas.TP = Ykk[0], Ykk[1]

surf.TP =Ykk[0], Ykk[1]
surf.coverages=Zkk
print('\nsurf.coverages after\n', surf.coverages)
saveInputsTChem()

Input directory :  /Users/odiazib/csp_clang_bld_develop/TChem++/master/src/example/SurfVerification/data/reaction-rates-surfaces/PT/

surf.coverages after
 [0.04281683 0.1546017  0.49482406 0.02760049 0.28015693]


In [5]:
runTchem() # run Tchem

In [6]:
print('*-------------*')
print('With an random guess')
compareTchemAndCantera()

*-------------*
With an random guess
-----------------------------
Gas phase
species name cantera  Tchem diff
-----------------------------
HE 0.0 0.0 0.0 0.0
    
AR 0.0 0.0 0.0 0.0
    
N2 0.0 0.0 0.0 0.0
    
O2 923899.4495440471 923913.9 -14.450455952901393 -1.5640723630729325e-05
    
H2 2789296.321182548 2789306.0 -9.678817451931536 -3.4699853789030603e-06
    
H2O 637290.1054091664 637302.2 -12.094590833527036 -1.8978155679604994e-05
    
H2O2 -178647.48511266516 -178649.2 1.714887334848754 -9.599280581908274e-06
    
CO 2401069.530697432 2401081.0 -11.46930256811902 -4.776747370904983e-06
    
CO2 175176.1785173294 175181.8 -5.621482670598198 -3.2090451556700044e-05
    
O -273686.0623771513 -273695.6 9.537622848700266 -3.484877076260101e-05
    
H -7633117.228172 -7633162.0 44.77182800043374 -5.865471033929847e-06
    
OH 9456254.259269428 9456332.0 -77.74073057249188 -8.221091400570905e-06
    
HO2 -5742769.204840772 -5742825.0 55.795159228146076 -9.715723762869396e-06
    
H

/Users/odiazib/opt/miniconda3/envs/cantera3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in double_scalars
/Users/odiazib/opt/miniconda3/envs/cantera3/lib/python3.7/site-packages/ipykernel_launcher.py:91: RuntimeWarning: invalid value encountered in double_scalars


In [7]:
# printRoP()

## Change gas temperature

In [8]:
Ykk[0] = 2000

gas.Y  = Ykk[2:24]
gas.TP = Ykk[0], Ykk[1]
surf.TP = Ykk[0], Ykk[1]

saveInputsTChem()


In [9]:
runTchem()

In [10]:
print('*-------------*')
print('Changing gas temperature to '+str(Ykk[0])+' [K]')
compareTchemAndCantera()

*-------------*
Changing gas temperature to 2000.0 [K]
-----------------------------
Gas phase
species name cantera  Tchem diff
-----------------------------
HE 0.0 0.0 0.0 0.0
    
AR 0.0 0.0 0.0 0.0
    
N2 0.0 0.0 0.0 0.0
    
O2 558441.8246877026 558450.5 -8.675312297418714 -1.5534854149347084e-05
    
H2 1592918.0806841296 1592923.0 -4.919315870385617 -3.0882415926077377e-06
    
H2O 469116.59491045756 469125.1 -8.505089542421047 -1.813001210081782e-05
    
H2O2 -177321.48524370213 -177323.2 1.714756297878921 -9.670324470394787e-06
    
CO 1499837.346523479 1499845.0 -7.653476520907134 -5.102871013745172e-06
    
CO2 101504.07951444254 101507.4 -3.3204855574585963 -3.2712828620707214e-05
    
O -171109.7593035681 -171115.5 5.740696431894321 -3.354978965115412e-05
    
H -4883597.6886812 -4883626.0 28.311318799853325 -5.7972258577873775e-06
    
OH 6592777.062044038 6592830.0 -52.93795596156269 -8.029689986991566e-06
    
HO2 -3877264.328026686 -3877301.0 36.67197331413627 -9.45820

/Users/odiazib/opt/miniconda3/envs/cantera3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in double_scalars
/Users/odiazib/opt/miniconda3/envs/cantera3/lib/python3.7/site-packages/ipykernel_launcher.py:91: RuntimeWarning: invalid value encountered in double_scalars


## Changing coverages

In [11]:
if make_random_sample:
    Ykk , Zkk = makeTChemRandomSample()
else:    
    Ykk , Zkk = getOriginalInput()

Zkk[-1] *=2
Zkk[-2] *=0.5

gas.Y  = Ykk[2:24]
gas.TP = Ykk[0], Ykk[1]
#surf.Y=Zkk[2:19]
surf.TP = Ykk[0], Ykk[1] 
surf.coverages = Zkk 

Zkk /= np.sum(Zkk)
print(Zkk)   

print('\nsurf.coverages after\n', surf.coverages)
saveInputsTChem()

[0.22929969 0.26403183 0.26496157 0.01748566 0.22422125]

surf.coverages after
 [0.22929969 0.26403183 0.26496157 0.01748566 0.22422125]


In [12]:
runTchem()
print('*-------------*')
print('Changing coverages ')
compareTchemAndCantera()

*-------------*
Changing coverages 
-----------------------------
Gas phase
species name cantera  Tchem diff
-----------------------------
HE 0.0 0.0 0.0 0.0
    
AR 0.0 0.0 0.0 0.0
    
N2 0.0 0.0 0.0 0.0
    
O2 1069122.767954189 1069141.0 -18.23204581090249 -1.7053276160033773e-05
    
H2 4098690.2674787794 4098736.0 -45.73252122057602 -1.1157837805760177e-05
    
H2O 819456.2889178266 819474.6 -18.311082173371688 -2.234540441144614e-05
    
H2O2 -489018.27331492724 -489026.1 7.826685072737746 -1.600489286357888e-05
    
CO 3017411.508718597 3017444.0 -32.49128140322864 -1.0767931821479233e-05
    
CO2 69291.18412762963 69293.84 -2.655872370363795 -3.832915260146023e-05
    
O 80788.29264244801 80789.76 -1.4673575519846054 -1.816299743427951e-05
    
H -11454239.433754534 -11454400.0 160.56624546647072 -1.401806260425267e-05
    
OH 13495115.87584124 13495320.0 -204.12415876053274 -1.5125780366654935e-05
    
HO2 -7812430.315403834 -7812554.0 123.68459616601467 -1.58317695227495e-05

/Users/odiazib/opt/miniconda3/envs/cantera3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in double_scalars
/Users/odiazib/opt/miniconda3/envs/cantera3/lib/python3.7/site-packages/ipykernel_launcher.py:91: RuntimeWarning: invalid value encountered in double_scalars


## Change gas mass fraction

In [13]:
if make_random_sample:
    Ykk , Zkk = makeTChemRandomSample()
else:    
    Ykk , Zkk = getOriginalInput()
    
    Ykk[-1] *=2
    Ykk[-2] *=0.5
    Ykk[-3] *=1.5 

gas.Y  = Ykk[2:24]
gas.TP = Ykk[0], Ykk[1]
#surf.Y=Zkk[2:19]
surf.TP = Ykk[0], Ykk[1]
surf.coverages = Zkk
#surf.set_unnormalized_coverages =  cover
print('\nsurf.coverages after\n', surf.coverages)
saveInputsTChem()


surf.coverages after
 [0.47036366 0.16031491 0.10025319 0.07391882 0.19514942]


In [14]:
runTchem()
print('*-------------*')
print('Changing molar gas fraction ')
compareTchemAndCantera()

*-------------*
Changing molar gas fraction 
-----------------------------
Gas phase
species name cantera  Tchem diff
-----------------------------
HE 0.0 0.0 0.0 0.0
    
AR 0.0 0.0 0.0 0.0
    
N2 0.0 0.0 0.0 0.0
    
O2 1212465.090282857 1212493.0 -27.9097171430476 -2.301898616853086e-05
    
H2 2009796.2510026281 2009817.0 -20.748997371876612 -1.0323930777324094e-05
    
H2O 731915.0225506672 731931.0 -15.977449332829565 -2.18296507662179e-05
    
H2O2 -194202.12870589882 -194205.1 2.9712941011821385 -1.5300007888594718e-05
    
CO 1072366.1490216055 1072376.0 -9.850978394504637 -9.186207904354658e-06
    
CO2 72628.41361377298 72631.12 -2.706386227015173 -3.726346332452378e-05
    
O -175709.6150469859 -175722.1 12.484953014092753 -7.105446683014014e-05
    
H -8805387.821360085 -8805495.0 107.17863991484046 -1.2171938600460825e-05
    
OH 12376087.93287271 12376250.0 -162.0671272892505 -1.3095182271513793e-05
    
HO2 -7520723.83857204 -7520833.0 109.16142795979977 -1.45147502159

/Users/odiazib/opt/miniconda3/envs/cantera3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in double_scalars
/Users/odiazib/opt/miniconda3/envs/cantera3/lib/python3.7/site-packages/ipykernel_launcher.py:91: RuntimeWarning: invalid value encountered in double_scalars
